In [ ]:
# Data handling
import pandas as pd
import numpy as np

# Spatial analysis
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# Biodiversity indices
from scipy.stats import entropy  # Shannon Index
from collections import Counter  # Hitung proporsi spesies

## **1. Import dan Eksplorasi Data**

Data yang digunakan dalam proyek ini berasal dari penelitian yang diselenggarakan oleh Elliot Morley untuk sensus burung laut yang berkembangbiak di Britania Raya dari tahun 1998 sampai 2002. Metode yang digunakan telah dijelaskan pada website BioTime sebagai berikut:

*Seabird Populations of Britain and Ireland summarises the results of Seabird 2000 ? a census ofall seabirds breeding in Britain and Ireland during 1998-2002. Seabird 2000 was launched on 12April 1999 by Elliot Morley MP and fieldwork was completed in 2002. Over 1000 surveyors tookpart and censused 3.200 colonies along 40.000 km of coastline and at 900 inland sites.Seabird 2000 followed on from two previous censuses: Operation Seafarer in 1969-70 and TheSeabird Colony Register in 1985-88. thus allowing population trends over 15-30 years to beassessed. In contrast to the previous two censuses. Seabird 2000 surveyed inland colonies ofseabirds. namely Black-headed Gulls. Common Gulls. Lesser Black-backed Gulls. GreatCormorants and Common Terns. Unit of abundance = AggregatedPresence, Unit of biomass = NA*

Sumber data: biotime.st-andrews.ac.uk/selectStudy.php?study=69

*   Jenis habitat: Pesisir (Marine)
*   Iklim: Sedang (Temperate)
*   Bioma: Ekoregion Laut
*   Garis tengah Lintang : 54.783267 LU
*   Garis tengah Bujur : -4.263030 BB
*   Durasi: 10 tahun, dari 1994 sampai 2003

In [ ]:
#Unggah Data
df = pd.read_csv('/content/raw_data_69.csv')

#Lihat 5 baris pertama data
df.head()

In [ ]:
#cek duplikat
df.duplicated().sum()

In [ ]:
#cek dataframe info
df.info()

Dari data yang tersedia sebanyak 11663 tidak ditemui data yang kosong, namun terdapat satu kolom yang dinilai tidak valid karena tidak berisi data, yaitu biomas. Maka kolom tersebut drop.

In [ ]:
#drop kolom biommas
df.drop(columns='BIOMAS', inplace=True)

# **Telaah Data**

Data terdiri dari berbagai jenis sampel populasi burung laut yang hidup di pesisir Britania Raya. Sampel data adalah banyaknya individu (abundance) saat pengamatan dilakukan.

In [ ]:
#daftar spesies yang teramati
df['valid_name'].unique()

In [ ]:
#jumlah kemunculan spesies
df['valid_name'].value_counts()


In [ ]:
#Statistik deskriptif tahun pengamatan
df['ABUNDANCE'].describe()

Berdasarkan analisis deskriptif, rata-rata pengamatan menjumpai 2 individu per pengamatan. Variasi pengamatan ada pada standar deviasi 1.85 yang menandai pengamatan jumlah individu antara 0.16 sampai 3.8 ekor. Namun terdapat outlier seperti jumlah pengamatan sebanyak 24 ekor, sebab beberapa wilayah terdiri dari koloni yang besar karena preferensi habitat.

# Tahun Pengamatan

In [ ]:
#Cek distribusi tahun pengamatan
df['YEAR'].value_counts().sort_index()

In [ ]:
import matplotlib.pyplot as plt

year_counts = df['YEAR'].value_counts().sort_index()
colors = ['red' if year in [1994, 1995, 1996, 1997, 2003] else 'skyblue' for year in year_counts.index]

plt.figure(figsize=(10, 5))
plt.bar(year_counts.index, year_counts.values, color=colors)
plt.xlabel('Tahun Pengamatan')
plt.ylabel('Jumlah Baris Data')
plt.title('Distribusi Tahun Pengamatan (Outlier Ditandai)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Terdapat outlier pada tahun 1994 sampai 1997 dan 2003. Pada tahun-tahun tersebut bisa saja muncul spesies tertentu yang muncul, sehingga outlier tidak dihapus.

# Lokasi Pengamatan

In [ ]:
#cek kordinat ekstrem
df[['LATITUDE', 'LONGITUDE']].agg(['min', 'max'])

In [ ]:
#Memperlihatkan area penelitian pada folium

# Titik ekstrem
north = df[df['LATITUDE'] == df['LATITUDE'].max()].iloc[0]
south = df[df['LATITUDE'] == df['LATITUDE'].min()].iloc[0]
east  = df[df['LONGITUDE'] == df['LONGITUDE'].max()].iloc[0]
west  = df[df['LONGITUDE'] == df['LONGITUDE'].min()].iloc[0]

# Peta dasar
center_lat = df['LATITUDE'].mean()
center_lon = df['LONGITUDE'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Tambahkan marker
folium.Marker([north['LATITUDE'], north['LONGITUDE']], popup='Paling Utara').add_to(m)
folium.Marker([south['LATITUDE'], south['LONGITUDE']], popup='Paling Selatan').add_to(m)
folium.Marker([east['LATITUDE'], east['LONGITUDE']], popup='Paling Timur').add_to(m)
folium.Marker([west['LATITUDE'], west['LONGITUDE']], popup='Paling Barat').add_to(m)

# Cakupan penelitian
lat_min = df['LATITUDE'].min()
lat_max = df['LATITUDE'].max()
lon_min = df['LONGITUDE'].min()
lon_max = df['LONGITUDE'].max()

rectangle_coords = [
    [lat_min, lon_min],  # Sudut kiri bawah
    [lat_min, lon_max],  # Sudut kanan bawah
    [lat_max, lon_max],  # Sudut kanan atas
    [lat_max, lon_min],  # Sudut kiri atas
    [lat_min, lon_min]   # Tutup loop ke titik awal
]

folium.PolyLine(locations=rectangle_coords, color='green', weight=2.5, opacity=0.8).add_to(m)

# Tampilkan peta
m


# Cetak Peta

In [ ]:
from shapely.geometry import Polygon
import geopandas as gpd
import matplotlib.pyplot as plt

# Ambil batas koordinat dari dataframe
lat_min, lat_max = df['LATITUDE'].min(), df['LATITUDE'].max()
lon_min, lon_max = df['LONGITUDE'].min(), df['LONGITUDE'].max()

# Buat polygon area penelitian (persegi)
rectangle = Polygon([
    (lon_min, lat_min),
    (lon_max, lat_min),
    (lon_max, lat_max),
    (lon_min, lat_max),
    (lon_min, lat_min)  # Tutup loop
])

# Buat GeoDataFrame untuk area
area_gdf = gpd.GeoDataFrame(index=[0], geometry=[rectangle], crs="EPSG:4326")

# Titik ekstrem
north = df[df['LATITUDE'] == lat_max].iloc[0]
south = df[df['LATITUDE'] == lat_min].iloc[0]
east  = df[df['LONGITUDE'] == lon_max].iloc[0]
west  = df[df['LONGITUDE'] == lon_min].iloc[0]

# Buat GeoDataFrame koloni
geometry = [Point(xy) for xy in zip(df['LONGITUDE'], df['LATITUDE'])]
colonies = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

fig, ax = plt.subplots(figsize=(10, 8))

# Plot area penelitian
area_gdf.boundary.plot(ax=ax, color='green', linewidth=2, label='Cakupan Penelitian')

# Plot titik ekstrem
for label, point, color in zip(
    ['Utara', 'Selatan', 'Timur', 'Barat'],
    [north, south, east, west],
    ['blue', 'red', 'orange', 'purple']
):
    gpd.GeoSeries([Point(point['LONGITUDE'], point['LATITUDE'])]).plot(ax=ax, color=color, markersize=50)
    ax.text(point['LONGITUDE'], point['LATITUDE'] + 0.2, label, fontsize=10, ha='center', color=color)

# Plot semua titik koloni
colonies.plot(ax=ax, color='gray', markersize=5, alpha=0.5, label='Koloni Burung Laut')

# Styling
ax.set_title("Peta Area Penelitian Burung Laut", fontsize=14)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.legend()
plt.tight_layout()
plt.savefig("peta_area_penelitian.png", dpi=300)
plt.show()


In [ ]:
from google.colab import files
files.download("peta_area_penelitian.png")

# Sampel Distribusi Spesies Terbanyak

In [ ]:
# Pengamatan spesies terbanyak
top_species = df['valid_name'].value_counts().nlargest(2).index.tolist()


In [ ]:
# Dataframe untuk 2 spesies teratas
df_top = df[df['valid_name'].isin(top_species)]

In [ ]:
# Melihat distribusi data pada sampel 2 spesies teratas

plt.figure(figsize=(8, 5))
sns.boxplot(data=df_top, x='valid_name', y='ABUNDANCE', palette='Set2')
plt.title('Distribusi Abundance untuk 2 Spesies Terbanyak')
plt.xlabel('Spesies')
plt.ylabel('Jumlah Individu per Pengamatan')
plt.tight_layout()
plt.show()


Berdasarkan visualisasi yang dilakukan sampel burung dengan pengamatan terbanyak, diperoleh data sebagai berikut:

*Fulmacus glacialis*
- Rata-rata pengamatan sekitar 2 ekor per pengamatan.
- Sebagian besar pengamatan mencatat 1 sampai 4 indivvidu.
- Outlier mencapai 24 individu, ada kemungkinan konsentrasi koloni sebagai habitat pilihan.

*Larus argentatus*
- Rata-rata pengamatan sekitar 1 ekor per pengamatan.
- Pengamatan sebagian besar hanya mencatat individu tunggal, bisa dikaitkan dengan perilaku spesies untuk hidup sendiri.
- Outlier sapai 10 individu pada lokasi tertentu namun jarang.

## **2. Klasifikasi Jenis Burung berdasarkan Lokasi Pesisir**

Unggah data vektor (shp) garis pantai untuk membandingkan seberapa banyak burung yang memiliki habitat kawasan daratan dan lautan. Data shapefile garis pesisir diunduh dari halaman www.naturalearthdata.com/downloads/10m-physical-vectors/10m-coastline/

In [ ]:
# Load data shapefile
coastline = gpd.read_file('/content/ne_10m_coastline.shp')

# Atur sistem proyeksi awal untuk mendeteksi wilayah penelitian
if coastline.crs is None:
    coastline.set_crs(epsg=4326, inplace=True)

# Ubah ke British National Grid
coastline = coastline.to_crs(epsg=27700)


# Konversi DataFrame ke GeoDataFrame

Dari data penelitian yang ada dalam tabel/dataframe, data tersebut ditransformasikan ke geodataframe untuk mengukur klasifikasi titik pengamatan.

In [ ]:
# Kolom geometry yang terdiri dari rekaman latitude dan longitude
geometry = [Point(xy) for xy in zip(df['LONGITUDE'], df['LATITUDE'])]

# Buat GeoDataFrame
colonies = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')  # proyeksi disesuaikan (CRS awal: WGS84)

Samakan sistem proyeksi yang sudah ditentukan, yaitu British National Grid EPS=27700)

In [ ]:
colonies = colonies.to_crs(epsg=27700)

Menghitung jarak ke garis pantai sesuai dengan file shp yang diunggah.

In [ ]:
# Fungsi untuk menghitung jarak minimum ke coastline
def min_distance_to_coast(point, coastline_gdf):
    return coastline_gdf.distance(point).min()

# Terapkan ke semua titik koloni
colonies['distance_to_coast_m'] = colonies.geometry.apply(lambda pt: min_distance_to_coast(pt, coastline))
colonies['distance_to_coast_km'] = colonies['distance_to_coast_m'] / 1000

# Klasifikasi Lokasi berdasarkan Jarak

Setelah data point ditransformasikan, data tersebut diklasifikasikan sesuai dengan jaraknya dari garis pesisir. Jika < 5 maka tergolong sebagai burung yang tinggal di kawasan pesisir, yaitu pertemuan antara daratan dan lautan.

In [ ]:
# Membuat klasifikasi berdasarkan jarak
colonies['lokasi'] = colonies['distance_to_coast_km'].apply(lambda x: 'zona transisi' if x < 5 else 'daratan atau laut lepas')

In [ ]:
#cek hasil
colonies[['valid_name', 'LATITUDE', 'LONGITUDE', 'distance_to_coast_km', 'lokasi']].head()

# Visualisasi dari Klasifikasi yang Dibuat

In [ ]:
# Peta Visualisasi klasifikasi lokasi
m = folium.Map(location=[df['LATITUDE'].mean(), df['LONGITUDE'].mean()], zoom_start=6)

for _, row in colonies.iterrows():
    color = 'blue' if row['lokasi'] == 'zona transisi' else 'green'
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=3,
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=f"{row['valid_name']} ({row['lokasi']})"
    ).add_to(m)

m

Burung yang teramati di zona transisi memiliki strategi untuk pemanfaatan habitat ganda, yaitu bersarang di darat dan mencari makan di laut. Selain jenis burung tersebut adalah jenis burung yang teramati jauh dari zona transisi, bisa di laut lepas ataupun daratan. Jenis burung ini memiliki daya jelajah yang luas dan spesialisasi habitat yang lebih sempit.

# Cetak Peta

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Buat GeoDataFrame dari df
geometry = [Point(xy) for xy in zip(df['LONGITUDE'], df['LATITUDE'])]
colonies = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Ubah ke proyeksi British National Grid (EPSG:27700)
colonies = colonies.to_crs(epsg=27700)
coastline = coastline.to_crs(epsg=27700)

# Hitung jarak ke garis pantai
def min_distance_to_coast(point, coastline_gdf):
    return coastline_gdf.distance(point).min()

colonies['distance_to_coast_m'] = colonies.geometry.apply(lambda pt: min_distance_to_coast(pt, coastline))
colonies['distance_to_coast_km'] = colonies['distance_to_coast_m'] / 1000

# Klasifikasi zona habitat
colonies['lokasi'] = colonies['distance_to_coast_km'].apply(
    lambda x: 'zona transisi' if x < 5 else 'daratan atau laut lepas'
)


In [ ]:
# Warna berdasarkan klasifikasi
color_map = {
    'zona transisi': 'blue',
    'daratan atau laut lepas': 'green'
}
colors = colonies['lokasi'].map(color_map)

# Plot peta
fig, ax = plt.subplots(figsize=(10, 8))

# Plot garis pantai dan koloni
coastline.plot(ax=ax, color='lightgray', linewidth=0.5)
colonies.plot(ax=ax, color=colors, markersize=5, alpha=0.6)

# Zoom ke area Britania Raya
ax.set_xlim(colonies.geometry.x.min() - 50000, colonies.geometry.x.max() + 50000)
ax.set_ylim(colonies.geometry.y.min() - 50000, colonies.geometry.y.max() + 50000)

# Tambahkan legenda manual
for label, color in color_map.items():
    ax.scatter([], [], c=color, label=label)
ax.legend(title="Zona Habitat")

# Styling
ax.set_title("Peta Statis Klasifikasi Lokasi Koloni Burung Laut", fontsize=14)
ax.set_xlabel("Koordinat X (meter)")
ax.set_ylabel("Koordinat Y (meter)")
plt.tight_layout()

# Simpan dan tampilkan
plt.savefig("peta_klasifikasi_lokasi.png", dpi=300)
plt.show()


In [ ]:
from google.colab import files
files.download("peta_klasifikasi_lokasi.png")

# Statistik Deskriptif

In [ ]:
colonies.groupby('lokasi')['ABUNDANCE'].describe()

Burung yang teramati pada zona transisi lebih banyak dengan rata-rata sebanyak 2.12 ekor diamati. Karakteristik ukuran pemusatan dan hampir sama, namun terdapat perbedaan standar deviasi. Pada burung laut di zona transisi sebesar 1.970974. Angka tersebut menandakan strategi koloni burung yang hampir mirip dengan data yang cukup terpusat.

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=colonies, x='lokasi', y='ABUNDANCE', palette='Set2')
plt.title('Distribusi Abundance berdasarkan Zona Habitat')
plt.xlabel('Kategori Habitat')
plt.ylabel('Jumlah Individu per Pengamatan')
plt.tight_layout()
plt.show()

Dari kedua jenis burung di lokasi pengamatan yang berbeda, tidak ada perbedaan yang signifikan antar habitat. Perbedaan ada pada outlier data yang bisa disimpulkan bahwa zona transisi menyediakan habitat yang lebih optimal dan koloni besar.

In [ ]:
from scipy.stats import mannwhitneyu

transisi = colonies[colonies['lokasi'] == 'zona transisi']['ABUNDANCE']
non_transisi = colonies[colonies['lokasi'] == 'daratan atau laut lepas']['ABUNDANCE']

stat, p = mannwhitneyu(transisi, non_transisi, alternative='two-sided')
print(f'Uji Mann-Whitney: Statistik={stat}, p-value={p}')


Berdasarkan uji statistik menggunakan Uji Mann-Whitney U, diperoleh habitat sangat berpengaruh pada jumlah individu yang teramati. Nilai p-value adalah 1.780513584651321e-19 yang sangat jauh di bawah p = 0.05.

## **3. Indeks Keragaman Spesies (Shannon Index dan Simpson Index)**

Ekosistem tidak terlepas dari keragaman unsur biotik dan abiotik penyusun lingkungan hidup. Semakin beragamnya ekosistem menandakan semakin kompleksnya sistem kehidupan yang membentuk hubungan antara makhluk hidup dengan lingkungan dan makhluk hidup dengan makhluk hidup lain.

# Shannon Index

In [ ]:
def shannon_index(df):
    total = df['ABUNDANCE'].sum()
    proporsi = df.groupby('valid_name')['ABUNDANCE'].sum() / total
    return entropy(proporsi, base=2)

# Data per zona
zona_transisi = colonies[colonies['lokasi'] == 'zona transisi']
zona_nontransisi = colonies[colonies['lokasi'] == 'daratan atau laut lepas']

# Hitung Shannon Index
shannon_transisi = shannon_index(zona_transisi)
shannon_nontransisi = shannon_index(zona_nontransisi)

print(f"Shannon Index Zona Transisi: {shannon_transisi:.3f}")
print(f"Shannon Index Daratan/Laut Lepas: {shannon_nontransisi:.3f}")

Kedua zona memiliki keragaman spesies yang tinggi, sebab Shannon Index mendekati nilai maksimum (rentang 0-4 dalam komunitas burung). Zona daratan/laut lepas memiliki skor yang sedikit lebih tinggi, menandakan sedikit lebih beragam. Namun perbedaan 0.076 sangatlah kecil, sehingga tidak signifikan secara ekologis.

# Simpson Index

In [ ]:
def simpson_index(df):
    total = df['ABUNDANCE'].sum()
    proporsi = df.groupby('valid_name')['ABUNDANCE'].sum() / total
    D = sum(proporsi**2)
    return 1 - D

# Hitung Simpson Index per zona
simpson_transisi = simpson_index(zona_transisi)
simpson_nontransisi = simpson_index(zona_nontransisi)

print(f"Simpson Index Zona Transisi: {simpson_transisi:.3f}")
print(f"Simpson Index Daratan/Laut Lepas: {simpson_nontransisi:.3f}")

Kedua zona memiliki komunitas spesies yang merata, tidak adanya satu jenis spesies yang lebih dominan. Perbedaan 0.004 dinilai sangat kecil antar zona, sehingga secara ekologis memiliki kemiripan dalam hal dominasi.

# Perbandingan

In [ ]:
# Data
zona = ['Zona Transisi', 'Daratan/Laut Lepas']
shannon = [3.908, 3.984]
simpson = [0.919, 0.923]

plt.figure(figsize=(8, 5))

# Plot Shannon Index
plt.scatter(zona, shannon, color='blue', s=100, label='Shannon Index', marker='o')
for i, val in enumerate(shannon):
    plt.text(zona[i], val + 0.02, f'{val:.3f}', ha='center', color='red', fontsize=10)

# Plot Simpson Index
plt.scatter(zona, simpson, color='green', s=100, label='Simpson Index', marker='^')
for i, val in enumerate(simpson):
    plt.text(zona[i], val + 0.02, f'{val:.3f}', ha='center', color='red', fontsize=10)

# Tampilan
plt.ylabel('Nilai Indeks')
plt.title('Dot Plot: Shannon & Simpson Index per Zona Habitat')
plt.ylim(0.9, 4.05)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()



Berdasarkan visualisasi data, pada zona transisi memiliki keragaman tinggi (Shannon index = 3.908), serta sedikit lebih padat dan terpusat (simpson index = 0.919). Sedangkan pada zona daratan/laut lepas juga memiliki keragaman tinggi (shannon index = 3.984), namun komunits lebih merata dan seimbang (simpson index = 0.923). Namun perbedaannya hanya sedikit sekali dan sulit untuk dibandingkan.

## **4. Visualisasi dan Interpretasi data pengamatan**

In [ ]:
from folium.plugins import MarkerCluster

# Peta dasar
m = folium.Map(location=[df['LATITUDE'].mean(), df['LONGITUDE'].mean()], zoom_start=6)

# Cluster marker
marker_cluster = MarkerCluster().add_to(m)

# Titik koloni
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=min(row['ABUNDANCE'], 10),  # batas radius
        color='crimson',
        fill=True,
        fill_opacity=0.6,
        popup=f"{row['valid_name']} ({row['ABUNDANCE']} individu)"
    ).add_to(marker_cluster)

m


# Cetak Peta

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Buat GeoDataFrame dari df
geometry = [Point(xy) for xy in zip(df['LONGITUDE'], df['LATITUDE'])]
colonies = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Proyeksi ke British National Grid
colonies = colonies.to_crs(epsg=27700)

# Ukuran marker berdasarkan abundance (dibatasi agar tidak terlalu besar)
sizes = colonies['ABUNDANCE'].apply(lambda x: min(x, 10)) * 5

# Plot peta
fig, ax = plt.subplots(figsize=(10, 8))
colonies.plot(ax=ax, markersize=sizes, color='crimson', alpha=0.6)

# Zoom ke Britania Raya
ax.set_xlim(colonies.geometry.x.min() - 50000, colonies.geometry.x.max() + 50000)
ax.set_ylim(colonies.geometry.y.min() - 50000, colonies.geometry.y.max() + 50000)

# Styling
ax.set_title("Peta Statis Koloni Burung Laut (Ukuran Marker = Abundance)", fontsize=14)
ax.set_xlabel("Koordinat X (meter)")
ax.set_ylabel("Koordinat Y (meter)")
plt.tight_layout()
plt.savefig("peta_marker_statis.png", dpi=300)
plt.show()


In [ ]:
# Hitung Shannon Index per lokasi
def shannon_per_lokasi(df):
    grouped = df.groupby(['LATITUDE', 'LONGITUDE'])
    hasil = []
    for (lat, lon), group in grouped:
        total = group['ABUNDANCE'].sum()
        proporsi = group.groupby('valid_name')['ABUNDANCE'].sum() / total
        shannon = entropy(proporsi, base=2)
        hasil.append({'LATITUDE': lat, 'LONGITUDE': lon, 'SHANNON': shannon})
    return pd.DataFrame(hasil)

df_shannon = shannon_per_lokasi(df)
top5 = df_shannon.sort_values(by='SHANNON', ascending=False).head(5)

# Peta titik dengan keragaman tertinggi
m2 = folium.Map(location=[df['LATITUDE'].mean(), df['LONGITUDE'].mean()], zoom_start=6)

for _, row in top5.iterrows():
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=f"Shannon Index: {row['SHANNON']:.3f}",
        icon=folium.Icon(color='green', icon='leaf')
    ).add_to(m2)

m2


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from scipy.stats import entropy

# Hitung Shannon Index per lokasi
def shannon_per_lokasi(df):
    grouped = df.groupby(['LATITUDE', 'LONGITUDE'])
    hasil = []
    for (lat, lon), group in grouped:
        total = group['ABUNDANCE'].sum()
        proporsi = group.groupby('valid_name')['ABUNDANCE'].sum() / total
        shannon = entropy(proporsi, base=2)
        hasil.append({'LATITUDE': lat, 'LONGITUDE': lon, 'SHANNON': shannon})
    return pd.DataFrame(hasil)

# Ambil 5 lokasi tertinggi
df_shannon = shannon_per_lokasi(df)
top5 = df_shannon.sort_values(by='SHANNON', ascending=False).head(5)

# Buat GeoDataFrame
geometry = [Point(xy) for xy in zip(top5['LONGITUDE'], top5['LATITUDE'])]
top5_gdf = gpd.GeoDataFrame(top5, geometry=geometry, crs="EPSG:4326").to_crs(epsg=27700)

# Pastikan coastline sudah dalam EPSG:27700
coastline = coastline.to_crs(epsg=27700)

# Plot peta
fig, ax = plt.subplots(figsize=(10, 8))

# Garis pantai
coastline.plot(ax=ax, color='lightgray', linewidth=0.5)

# Zoom ke area Britania Raya
ax.set_xlim(top5_gdf.geometry.x.min() - 50000, top5_gdf.geometry.x.max() + 50000)
ax.set_ylim(top5_gdf.geometry.y.min() - 50000, top5_gdf.geometry.y.max() + 50000)

# Titik Shannon tertinggi
top5_gdf.plot(ax=ax, color='darkgreen', markersize=80)

# Label nilai Shannon
for idx, row in top5_gdf.iterrows():
    ax.text(row.geometry.x + 1000, row.geometry.y + 1000,
            f"{row['SHANNON']:.2f}", fontsize=10, color='black')

# Styling
ax.set_title("🟢 Lokasi dengan Shannon Index Tertinggi", fontsize=14)
ax.set_xlabel("Koordinat X (meter)")
ax.set_ylabel("Koordinat Y (meter)")
plt.tight_layout()
plt.savefig("peta_shannon_coastline.png", dpi=300)
plt.show()

